In [114]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn.ensemble import RandomForestClassifier

In [84]:
df = pd.read_excel('/content/churn_data_case.xlsx')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [85]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7032.000000,7032.000000,7032.000000,7032.000000
mean,0.162400,32.421786,64.798208,2283.300441
std,0.368844,24.545260,30.085974,2266.771362
min,0.000000,1.000000,18.250000,18.800000
25%,0.000000,9.000000,35.587500,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.862500,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


In [86]:
df['SeniorCitizen'].value_counts()

0    5890
1    1142
Name: SeniorCitizen, dtype: int64

In [87]:
df['SeniorCitizen'] = df['SeniorCitizen'].astype('object')

In [88]:
x = df.drop(['Churn','customerID'],axis=1)
y = df['Churn']

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7032 non-null   object 
 1   gender            7032 non-null   object 
 2   SeniorCitizen     7032 non-null   object 
 3   Partner           7032 non-null   object 
 4   Dependents        7032 non-null   object 
 5   tenure            7032 non-null   int64  
 6   PhoneService      7032 non-null   object 
 7   MultipleLines     7032 non-null   object 
 8   InternetService   7032 non-null   object 
 9   OnlineSecurity    7032 non-null   object 
 10  OnlineBackup      7032 non-null   object 
 11  DeviceProtection  7032 non-null   object 
 12  TechSupport       7032 non-null   object 
 13  StreamingTV       7032 non-null   object 
 14  StreamingMovies   7032 non-null   object 
 15  Contract          7032 non-null   object 
 16  PaperlessBilling  7032 non-null   object 


In [90]:
le = LabelEncoder()
x['MultipleLines'] = le.fit_transform(x['MultipleLines'])

In [91]:
x['InternetService'] = le.fit_transform(x['InternetService'])

In [92]:
x['OnlineBackup'] = le.fit_transform(x['OnlineBackup'])

In [93]:
x['Contract'] = le.fit_transform(x['Contract'])

In [94]:
x['PaymentMethod'] = le.fit_transform(x['PaymentMethod'])

In [95]:
dummies = x[['gender','Partner','Dependents','PhoneService','OnlineSecurity','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling']]

In [96]:
dummies = pd.get_dummies(dummies)

In [97]:
x.drop(['gender','Partner','Dependents','PhoneService','OnlineSecurity','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling'],axis=1,inplace=True)

In [98]:
x = pd.concat([x,dummies],axis=1)

In [99]:
x.head()

,SeniorCitizen,tenure,MultipleLines,InternetService,OnlineBackup,Contract,PaymentMethod,MonthlyCharges,TotalCharges,gender_Female,...,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,PaperlessBilling_No,PaperlessBilling_Yes
0,0,1,1,0,2,0,2,29.85,29.85,1,...,0,0,1,0,0,1,0,0,0,1
1,0,34,0,0,0,1,3,56.95,1889.50,0,...,0,0,1,0,0,1,0,0,1,0
2,0,2,0,0,2,0,3,53.85,108.15,0,...,0,0,1,0,0,1,0,0,0,1
3,0,45,1,0,0,1,0,42.30,1840.75,0,...,0,1,1,0,0,1,0,0,1,0
4,0,2,0,1,0,0,2,70.70,151.65,1,...,0,0,1,0,0,1,0,0,0,1


In [100]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=23)

# USANDO REDE NEURAL

In [101]:
deep_model = MLPClassifier(activation='identity',hidden_layer_sizes=(100,50,10)).fit(x_train,y_train)

In [102]:
y_pred = deep_model.predict(x_test)

In [105]:
print('Accuracy: ', accuracy_score(y_pred,y_test))
print('F1-score: ', f1_score(pos_label='Yes',y_pred=y_pred,y_true=y_test))

Accuracy:  0.7725657427149965
F1-score:  0.4181818181818182


# USANDO RANDOM FOREST

In [107]:
rf_model = RandomForestClassifier(n_estimators=1000).fit(x_train,y_train)

In [110]:
y_pred = rf_model.predict(x_test)

In [111]:
print('Accuracy: ', accuracy_score(y_pred,y_test))
print('F1-score: ', f1_score(pos_label='Yes',y_pred=y_pred,y_true=y_test))

Accuracy:  0.7732764747690121
F1-score:  0.5173978819969742


# REALIZANDO O TUNNING

In [112]:
param = {
    'max_depth':[1,2,3,4,5,6,7,8,9,10],
    'n_estimators':[100,300,500]
}

In [116]:
grid = GridSearchCV(rf_model,param,scoring='roc_auc',n_jobs=-1).fit(x_train,y_train)

In [117]:
grid.best_estimator_.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 7,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

# CRIANDO O MODELO TUNADO

In [118]:
tuned_rf = RandomForestClassifier(bootstrap= True,
 ccp_alpha= 0.0,
 class_weight=None,
 criterion= 'gini',
 max_depth= 7,
 max_features= 'sqrt',
 max_leaf_nodes= None,
 max_samples= None,
 min_impurity_decrease= 0.0,
 min_samples_leaf= 1,
 min_samples_split= 2,
 min_weight_fraction_leaf= 0.0,
 n_estimators= 100,
 n_jobs=None,
 oob_score= False,
 random_state= None,
 verbose= 0,
 warm_start= False)

In [119]:
tuned_rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=7)

In [120]:
y_pred = tuned_rf.predict(x_test)

In [121]:
print('Accuracy: ', accuracy_score(y_pred,y_test))
print('F1-score: ', f1_score(pos_label='Yes',y_pred=y_pred,y_true=y_test))

Accuracy:  0.7889125799573561
F1-score:  0.533751962323391
